In [1]:
import pandas as pd
import missingno as msno
import chart_studio.plotly as py
import cufflinks as cf
import plotly.graph_objects as go
cf.go_offline(connected=True)
from sklearn import preprocessing

In [2]:
# 변수정리

# df -> 통합 데이터
# product_order_month_df -> 월별 카테고리 데이터
# product_order_month_max_count -> 월별 카테고리 best건수 카테고리
# product_order_month_max_price -> 월별 카테고리 best매출 카테고리
# month_payment_df -> 월별 매출 df
# payment_type -> 지불 방법 df


### 파일로드

In [3]:
customers = pd.read_csv("data/olist_customers_dataset.csv", encoding="utf-8")
categorys = pd.read_csv("data/product_category_name_translation.csv", encoding="utf-8")
products = pd.read_csv("data/olist_products_dataset.csv", encoding="utf-8")
orders = pd.read_csv("data/olist_orders_dataset.csv", encoding="utf-8")
order_items = pd.read_csv("data/olist_order_items_dataset.csv", encoding="utf-8")
payments = pd.read_csv("data/olist_order_payments_dataset.csv", encoding="utf-8")
reviews = pd.read_csv("data/olist_order_reviews_dataset.csv", encoding="utf-8")
sellers = pd.read_csv("data/olist_sellers_dataset.csv", encoding="utf-8")
geolocation = pd.read_csv("data/olist_geolocation_dataset.csv", encoding="utf-8")

### 데이터 전처리 & 통합

In [4]:
customers1=customers[["customer_id","customer_city","customer_state"]]
customers1.head(1)

,customer_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP


In [5]:
products1 = products[["product_id","product_category_name"]]
products1 = products1.dropna()
products1.head(1)

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria


In [6]:
orders1=orders[["order_id","customer_id","order_purchase_timestamp","order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"]]
orders1 = orders1.dropna()
orders1.head(1)

,order_id,customer_id,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00


In [7]:
reviews1 = reviews[["order_id","review_score"]]
reviews1 = reviews1.drop_duplicates("order_id",keep="first")
reviews1.head(1)

,order_id,review_score
0,73fc7af87114b39712e6da79b0a377eb,4


In [8]:
order_items1 = order_items.drop_duplicates("order_id", keep="first")
order_items1= order_items1[["order_id","product_id"]]
order_items1.head(1)

,order_id,product_id
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61


In [9]:
payments1 = payments.groupby("order_id")["payment_value"].sum().reset_index()
payments1.head(1)

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19


In [10]:
#### merge

In [11]:
# product1 & category merge
products_name = pd.merge(products1, categorys, on="product_category_name")
products_name = products_name.drop(columns="product_category_name")
products_name.head(1)

,product_id,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery


In [12]:
# produst_name & order_items1 merge
order_products_name = pd.merge(order_items1, products_name, on="product_id")
order_products_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97229 entries, 0 to 97228
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       97229 non-null  object
 1   product_id                     97229 non-null  object
 2   product_category_name_english  97229 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


In [13]:
#customer1 & order1 merge
customer_order = pd.merge(customers1, orders1, on="customer_id")
customer_order.head(1)

,customer_id,customer_city,customer_state,order_id,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00


In [14]:
# customer_order & review merge
customer_order_review = pd.merge(customer_order, reviews1, on="order_id")
customer_order_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96475 entries, 0 to 96474
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   customer_id                    96475 non-null  object
 1   customer_city                  96475 non-null  object
 2   customer_state                 96475 non-null  object
 3   order_id                       96475 non-null  object
 4   order_purchase_timestamp       96475 non-null  object
 5   order_delivered_carrier_date   96475 non-null  object
 6   order_delivered_customer_date  96475 non-null  object
 7   order_estimated_delivery_date  96475 non-null  object
 8   review_score                   96475 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 7.4+ MB


In [15]:
# customer_order_review & payment1 merge
customer_order_review_payment = pd.merge(customer_order_review, payments1, on="order_id")
customer_order_review_payment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96474 entries, 0 to 96473
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_id                    96474 non-null  object 
 1   customer_city                  96474 non-null  object 
 2   customer_state                 96474 non-null  object 
 3   order_id                       96474 non-null  object 
 4   order_purchase_timestamp       96474 non-null  object 
 5   order_delivered_carrier_date   96474 non-null  object 
 6   order_delivered_customer_date  96474 non-null  object 
 7   order_estimated_delivery_date  96474 non-null  object 
 8   review_score                   96474 non-null  int64  
 9   payment_value                  96474 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 8.1+ MB


In [16]:
# customer_order_review_payment & order_products_name merge
df = pd.merge(order_products_name, customer_order_review_payment, on="order_id")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95096 entries, 0 to 95095
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       95096 non-null  object 
 1   product_id                     95096 non-null  object 
 2   product_category_name_english  95096 non-null  object 
 3   customer_id                    95096 non-null  object 
 4   customer_city                  95096 non-null  object 
 5   customer_state                 95096 non-null  object 
 6   order_purchase_timestamp       95096 non-null  object 
 7   order_delivered_carrier_date   95096 non-null  object 
 8   order_delivered_customer_date  95096 non-null  object 
 9   order_estimated_delivery_date  95096 non-null  object 
 10  review_score                   95096 non-null  int64  
 11  payment_value                  95096 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usa

In [17]:
# 구매일 datetime 형변환
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"], format="%Y-%m-%d %H:%M:%S", errors="raise")

In [18]:
payment2 = payments[["order_id","payment_type"]]
payment2 = payment2.drop_duplicates("order_id", keep="first")

In [19]:
# 최종데이터 df
df = pd.merge(df, payment2, on="order_id")
df.head()

,order_id,product_id,product_category_name_english,customer_id,customer_city,customer_state,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_score,payment_value,payment_type
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,3ce436f183e68e07877b285a838db11a,campos dos goytacazes,RJ,2017-09-13 08:59:02,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,5,72.19,credit_card
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,e6eecc5a77de221464d1c4eaff0a9b64,jatai,GO,2017-06-28 11:52:20,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26 00:00:00,5,73.86,boleto
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,4ef55bf80f711b372afebcb7c715344a,belo horizonte,MG,2018-05-18 10:25:53,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07 00:00:00,4,83.23,credit_card
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,30407a72ad8b3f4df4d15369126b20c9,sao jose dos pinhais,PR,2017-08-01 18:38:42,2017-08-02 19:07:36,2017-08-09 21:26:33,2017-08-25 00:00:00,5,75.07,credit_card
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,91a792fef70ecd8cc69d3c7feb3d12da,conselheiro lafaiete,MG,2017-08-10 21:48:40,2017-08-11 19:43:07,2017-08-24 20:04:21,2017-09-01 00:00:00,5,72.19,credit_card


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95096 entries, 0 to 95095
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       95096 non-null  object        
 1   product_id                     95096 non-null  object        
 2   product_category_name_english  95096 non-null  object        
 3   customer_id                    95096 non-null  object        
 4   customer_city                  95096 non-null  object        
 5   customer_state                 95096 non-null  object        
 6   order_purchase_timestamp       95096 non-null  datetime64[ns]
 7   order_delivered_carrier_date   95096 non-null  object        
 8   order_delivered_customer_date  95096 non-null  object        
 9   order_estimated_delivery_date  95096 non-null  object        
 10  review_score                   95096 non-null  int64         
 11  payment_value  

In [21]:
df.describe()

,review_score,payment_value
count,95096.000000,95096.000000
mean,4.144591,160.033276
std,1.294347,218.743673
min,1.000000,9.590000
25%,4.000000,62.010000
50%,5.000000,105.220000
75%,5.000000,176.880000
max,5.000000,13664.080000


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">지불 방법</font><br>
</div>

In [22]:
payment_type = df[["payment_type","order_id","payment_value"]].copy()
payment_type_count = payment_type.groupby("payment_type")["order_id"].size().sort_values(ascending=False)
payment_type_count

payment_type
credit_card    72177
boleto         18925
voucher         2540
debit_card      1454
Name: order_id, dtype: int64

In [23]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Pie(    
        labels=payment_type_count.index, values=payment_type_count.values,
        textinfo='label+percent', insidetextorientation='horizontal'
    )
)

fig.update_layout(
    {
        "title": {
            "text": "Payment Type(count)",
            "x": 0.53,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True
    }
)

fig.show()

In [24]:
#sales
payment_type_sales = payment_type.groupby("payment_type")["payment_value"].sum().sort_values(ascending=False)
payment_type_sales

payment_type
credit_card    11968179.55
boleto          2729352.36
voucher          316653.37
debit_card       204339.15
Name: payment_value, dtype: float64

In [25]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Pie(    
        labels=payment_type_sales.index, values=payment_type_sales.values,
        textinfo='label+percent', insidetextorientation='horizontal'
    )
)

fig.update_layout(
    {
        "title": {
            "text": "Payment Type(sales)",
            "x": 0.53,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True
    }
)

fig.show()

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">월별 매출액</font><br>
</div>

In [26]:
# 필요한 컬럼 뽑아내기
month_payment_df = df[["order_purchase_timestamp","payment_value"]].copy()
month_payment_df = month_payment_df.groupby(pd.Grouper(key='order_purchase_timestamp', freq ="M")).sum()
month_payment_df.head()

,payment_value
order_purchase_timestamp,
2016-10-31,47175.28
2016-11-30,0.00
2016-12-31,19.62
2017-01-31,125746.94
2017-02-28,261752.64


In [27]:
#시각화

In [28]:
colors = ['#03588C',] * len(month_payment_df.index)
colors[13] = '#F24472'

In [29]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=month_payment_df.index, 
        y=month_payment_df['payment_value'], 
        text=month_payment_df['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:,.0f}',
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Montly Sales</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Aug. 2018",
            "showticklabels":True,
            "tick0": "2016-10-31",
            "dtick": "M1",
            "tickfont": {
                "size": 8                
            }
        },
        "yaxis": {
            "title": "sales",
            "tickfont": {
                "size": 10                
            }
        },
        "template":'plotly_white'
    },
fig.add_annotation(
            x="2017-11-30",
            y=1157647,
            text="<b>2017.11</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77CFD9",
            ax=20,
            ay=-30,
            bordercolor="#77CFD9",
            borderwidth=2,
            borderpad=6,
            bgcolor="#F25D50",
            opacity=0.9
)
)

fig.show()

In [30]:
# 일별
payment_day_df = df[["order_purchase_timestamp","payment_value"]].copy()
payment_day_df = payment_day_df.groupby(pd.Grouper(key='order_purchase_timestamp', freq ="D")).sum()
payment_day_df.head()

,payment_value
order_purchase_timestamp,
2016-10-03,595.14
2016-10-04,9755.92
2016-10-05,7209.50
2016-10-06,6733.52
2016-10-07,7096.69


In [31]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=payment_day_df.index, 
        y=payment_day_df['payment_value'], 
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Daily Sales</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Aug. 2018",
            "showticklabels":True,
            "tick0": "2016-10-01",
            "dtick": "M1",
            "tickfont": {
                "size": 10                
            }
        },
        "yaxis": {
            "title": "sales",
            "tickfont": {
                "size": 10                
            }
        },
        "template":'plotly_white'
    },
fig.add_annotation(
            x="2017-11-25",
            y=177000,
            text="<b>2017.11.24</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77CFD9",
            ax=20,
            ay=-30,
            bordercolor="#77CFD9",
            borderwidth=2,
            borderpad=6,
            bgcolor="#F25D50",
            opacity=0.9
)
)

fig.show()

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">카테고리 분석</font><br>
</div>

In [32]:
category_values = df[["product_category_name_english","payment_value"]].copy()
category_values = category_values.groupby("product_category_name_english")["payment_value"].sum().sort_values(ascending=False)
category_values = pd.DataFrame(category_values)
category_values

,payment_value
product_category_name_english,
health_beauty,1410890.88
watches_gifts,1261634.89
bed_bath_table,1224487.19
sports_leisure,1119196.38
computers_accessories,1030732.32
...,...
flowers,1794.87
home_comfort_2,1131.41
cds_dvds_musicals,954.99


In [33]:
category_values.iplot(kind="bar")

In [34]:
#시각화

In [35]:
category_values_top20 = category_values.head(20)

In [36]:
colors1 = ['#03588C',] * len(month_payment_df.index)

In [37]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=category_values_top20.index, 
        y=category_values_top20['payment_value'], 
        text=category_values_top20['payment_value'], 
        textposition='auto', 
        texttemplate='R$ %{text:,.0f}',
        marker_color=colors1
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Top20 Category</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Aug. 2018",
            "showticklabels":True,
            "tickfont": {
                "size": 10                
            }
        },
        "yaxis": {
            "title": "Sales",
            "tickfont": {
                "size": 10                
            }
        },
        "template":'plotly_white'
    }
)

fig.show()

In [38]:
# health_beauty 매출에서 차지하는 비율
category_values = category_values.reset_index()
round(category_values.loc[0, "payment_value"] / category_values["payment_value"].sum() * 100, 2)

9.27

In [39]:
# pet_shop
category_values = category_values.reset_index()
round(category_values.loc[16, "payment_value"] / category_values["payment_value"].sum() * 100, 2)

1.65

In [40]:
# top 20의 매출 비율
round(category_values["payment_value"][:20].sum() / category_values["payment_value"].sum() * 100, 2)

85.06

In [41]:
# top 5의 매출 비율
round(category_values["payment_value"][:5].sum() / category_values["payment_value"].sum() * 100, 2)

39.73

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">카테고리 Top20 비교분석</font><br>
</div>

In [42]:
df2 = df[["product_category_name_english","order_purchase_timestamp","payment_value"]].copy()
df2["year"] = df2["order_purchase_timestamp"].dt.year #year 컬럼추가하기
df2["month"] = df2["order_purchase_timestamp"].dt.month #month 컬럼 추가하기
df2.head()

,product_category_name_english,order_purchase_timestamp,payment_value,year,month
0,cool_stuff,2017-09-13 08:59:02,72.19,2017,9
1,cool_stuff,2017-06-28 11:52:20,73.86,2017,6
2,cool_stuff,2018-05-18 10:25:53,83.23,2018,5
3,cool_stuff,2017-08-01 18:38:42,75.07,2017,8
4,cool_stuff,2017-08-10 21:48:40,72.19,2017,8


In [43]:
# 2017 & 2018 top20 데이터 추이

In [44]:
df_all = df2[df2["year"]!=2016]
df_all

,product_category_name_english,order_purchase_timestamp,payment_value,year,month
0,cool_stuff,2017-09-13 08:59:02,72.19,2017,9
1,cool_stuff,2017-06-28 11:52:20,73.86,2017,6
2,cool_stuff,2018-05-18 10:25:53,83.23,2018,5
3,cool_stuff,2017-08-01 18:38:42,75.07,2017,8
4,cool_stuff,2017-08-10 21:48:40,72.19,2017,8
...,...,...,...,...,...
95091,garden_tools,2018-07-13 20:04:05,181.10,2018,7
95092,furniture_decor,2018-08-18 10:00:59,112.52,2018,8
95093,watches_gifts,2017-06-01 16:53:03,756.91,2017,6
95094,sports_leisure,2017-12-18 16:33:07,273.92,2017,12


In [45]:
# 2017 & 2018 top20 항목만 뽑아내기
df_all_top20 = df_all.groupby("product_category_name_english")["payment_value"].sum().sort_values(ascending=False).reset_index()[:20]
df_all_top20 = list(df_all_top20["product_category_name_english"])
df_all_top20

['health_beauty',
 'watches_gifts',
 'bed_bath_table',
 'sports_leisure',
 'computers_accessories',
 'furniture_decor',
 'housewares',
 'cool_stuff',
 'auto',
 'garden_tools',
 'toys',
 'baby',
 'perfumery',
 'telephony',
 'office_furniture',
 'stationery',
 'pet_shop',
 'computers',
 'musical_instruments',
 'electronics']

In [46]:
df_all_20 = []
for rank in df_all_top20:  #top20 항목 데이터프레임 만들기
    df3 = df_all[df_all["product_category_name_english"] == rank]
    df_all_20.append(df3)

df_all = pd.concat(df_all_20).reset_index(drop=True)
df_all = df_all.groupby(["product_category_name_english", pd.Grouper(key="order_purchase_timestamp", freq="M")])["payment_value"].sum() # 월별 카테고리
df_all = pd.DataFrame(df_all).reset_index()
df_all = df_all.pivot_table("payment_value", "order_purchase_timestamp", "product_category_name_english", aggfunc="sum") #pivot_table
df_all

product_category_name_english,auto,baby,bed_bath_table,computers,computers_accessories,cool_stuff,electronics,furniture_decor,garden_tools,health_beauty,housewares,musical_instruments,office_furniture,perfumery,pet_shop,sports_leisure,stationery,telephony,toys,watches_gifts
order_purchase_timestamp,,,,,,,,,,,,,,,,,,,,
2017-01-31,5386.38,6941.93,4911.83,2217.99,4377.60,7603.02,772.43,15597.48,8448.34,13802.80,3269.76,357.74,1341.78,5654.01,996.39,10754.92,1754.61,1145.89,5302.23,8310.83
2017-02-28,14264.10,3544.53,18263.15,3602.47,13014.71,13946.98,1577.41,25167.08,13411.85,25133.55,12510.25,2429.39,12857.49,8825.04,4361.43,18533.57,2646.03,8369.64,10359.95,11524.20
2017-03-31,15304.25,4358.31,30112.54,NaN,31286.73,22623.77,4655.16,27905.47,26706.98,29463.99,16560.87,1781.43,13152.68,21300.56,9008.30,29284.00,6994.89,10366.15,14248.29,26092.77
2017-04-30,17205.68,4690.33,28842.93,NaN,20703.80,23395.72,5586.08,23510.63,19853.86,25498.75,15724.27,4114.27,11164.03,21124.96,7129.66,26977.70,5119.99,9305.75,15072.16,24958.48
2017-05-31,20227.30,11019.20,38989.73,NaN,50459.96,37350.47,7299.63,27852.37,24002.57,52254.72,26442.80,10102.98,10019.71,22237.26,9706.52,39530.31,6664.42,12051.75,25370.33,38960.51
2017-06-30,31621.25,9229.24,40151.50,NaN,40691.12,42539.56,2675.70,21512.54,18163.41,36887.18,29079.45,5515.32,9095.80,26386.33,6227.04,38038.22,4176.05,8856.83,18073.34,29223.36
2017-07-31,16412.42,17954.75,73665.21,NaN,44059.87,41611.19,8661.88,29896.96,22942.30,39599.66,24665.05,8030.27,21022.54,14571.75,8252.05,43975.35,8087.37,11296.99,27690.60,35835.46
2017-08-31,22138.37,21953.81,65486.93,33497.41,39678.82,43826.85,3828.74,42073.93,27272.36,55454.46,28530.89,7822.98,12402.25,14956.67,12338.85,48406.79,8087.84,12791.05,20529.72,39526.36
2017-09-30,16412.01,27086.63,62014.85,52948.15,33240.99,41594.55,5557.35,37907.00,26543.38,58236.94,25137.70,9149.88,11401.98,16265.37,9875.16,57334.05,7202.08,21184.66,35123.27,48800.94


In [47]:
# 2017년
df_2017 = df2[df2["year"]==2017]
df_2017

,product_category_name_english,order_purchase_timestamp,payment_value,year,month
0,cool_stuff,2017-09-13 08:59:02,72.19,2017,9
1,cool_stuff,2017-06-28 11:52:20,73.86,2017,6
3,cool_stuff,2017-08-01 18:38:42,75.07,2017,8
4,cool_stuff,2017-08-10 21:48:40,72.19,2017,8
5,cool_stuff,2017-07-27 15:11:51,82.83,2017,7
...,...,...,...,...,...
95085,computers_accessories,2017-02-10 14:49:23,317.79,2017,2
95088,toys,2017-08-31 21:15:02,69.58,2017,8
95090,home_construction,2017-03-15 17:16:36,162.10,2017,3
95093,watches_gifts,2017-06-01 16:53:03,756.91,2017,6


In [48]:
# 2017 top20 항목만 뽑아내기
df_2017_top20 = df_2017.groupby("product_category_name_english")["payment_value"].sum().sort_values(ascending=False).reset_index()[:20]
df_2017_top20 = list(df_2017_top20["product_category_name_english"])
df_2017_top20

['bed_bath_table',
 'health_beauty',
 'sports_leisure',
 'watches_gifts',
 'computers_accessories',
 'cool_stuff',
 'furniture_decor',
 'toys',
 'garden_tools',
 'housewares',
 'auto',
 'perfumery',
 'baby',
 'telephony',
 'computers',
 'office_furniture',
 'stationery',
 'small_appliances',
 'pet_shop',
 'fashion_bags_accessories']

In [49]:
df_2017_20 = []
for rank in df_2017_top20:  #top20 항목 데이터프레임 만들기
    df3 = df_2017[df_2017["product_category_name_english"] == rank]
    df_2017_20.append(df3)

df_2017 = pd.concat(df_2017_20).reset_index(drop=True)
df_2017 = df_2017.groupby(["product_category_name_english", pd.Grouper(key="order_purchase_timestamp", freq="M")])["payment_value"].sum() # 월별 카테고리
df_2017 = pd.DataFrame(df_2017).reset_index()
df_2017 = df_2017.pivot_table("payment_value", "order_purchase_timestamp", "product_category_name_english", aggfunc="sum") #pivot_table
df_2017

product_category_name_english,auto,baby,bed_bath_table,computers,computers_accessories,cool_stuff,fashion_bags_accessories,furniture_decor,garden_tools,health_beauty,housewares,office_furniture,perfumery,pet_shop,small_appliances,sports_leisure,stationery,telephony,toys,watches_gifts
order_purchase_timestamp,,,,,,,,,,,,,,,,,,,,
2017-01-31,5386.38,6941.93,4911.83,2217.99,4377.60,7603.02,2136.61,15597.48,8448.34,13802.80,3269.76,1341.78,5654.01,996.39,1788.25,10754.92,1754.61,1145.89,5302.23,8310.83
2017-02-28,14264.10,3544.53,18263.15,3602.47,13014.71,13946.98,2793.37,25167.08,13411.85,25133.55,12510.25,12857.49,8825.04,4361.43,8022.25,18533.57,2646.03,8369.64,10359.95,11524.20
2017-03-31,15304.25,4358.31,30112.54,NaN,31286.73,22623.77,5789.49,27905.47,26706.98,29463.99,16560.87,13152.68,21300.56,9008.30,12033.13,29284.00,6994.89,10366.15,14248.29,26092.77
2017-04-30,17205.68,4690.33,28842.93,NaN,20703.80,23395.72,3878.33,23510.63,19853.86,25498.75,15724.27,11164.03,21124.96,7129.66,24045.81,26977.70,5119.99,9305.75,15072.16,24958.48
2017-05-31,20227.30,11019.20,38989.73,NaN,50459.96,37350.47,8719.31,27852.37,24002.57,52254.72,26442.80,10019.71,22237.26,9706.52,5727.35,39530.31,6664.42,12051.75,25370.33,38960.51
2017-06-30,31621.25,9229.24,40151.50,NaN,40691.12,42539.56,7989.40,21512.54,18163.41,36887.18,29079.45,9095.80,26386.33,6227.04,10342.28,38038.22,4176.05,8856.83,18073.34,29223.36
2017-07-31,16412.42,17954.75,73665.21,NaN,44059.87,41611.19,6315.45,29896.96,22942.30,39599.66,24665.05,21022.54,14571.75,8252.05,4310.00,43975.35,8087.37,11296.99,27690.60,35835.46
2017-08-31,22138.37,21953.81,65486.93,33497.41,39678.82,43826.85,7807.01,42073.93,27272.36,55454.46,28530.89,12402.25,14956.67,12338.85,4838.18,48406.79,8087.84,12791.05,20529.72,39526.36
2017-09-30,16412.01,27086.63,62014.85,52948.15,33240.99,41594.55,13768.13,37907.00,26543.38,58236.94,25137.70,11401.98,16265.37,9875.16,10913.08,57334.05,7202.08,21184.66,35123.27,48800.94


In [50]:
#2018(1~8)
df_2018 = df2[df2["year"]==2018]
df_2018.head()

,product_category_name_english,order_purchase_timestamp,payment_value,year,month
2,cool_stuff,2018-05-18 10:25:53,83.23,2018,5
6,cool_stuff,2018-03-18 21:00:44,103.40,2018,3
9,furniture_decor,2018-01-14 14:33:31,216.87,2018,1
10,furniture_decor,2018-01-06 14:03:53,216.87,2018,1
11,perfumery,2018-08-08 10:00:35,25.78,2018,8


In [51]:
# 2018 top20만 뽑아내기
df_2018_top20 = df_2018.groupby("product_category_name_english")["payment_value"].sum().sort_values(ascending=False).reset_index()[:20]
df_2018_top20 = list(df_2018_top20["product_category_name_english"])
df_2018_top20

['health_beauty',
 'watches_gifts',
 'bed_bath_table',
 'sports_leisure',
 'computers_accessories',
 'housewares',
 'furniture_decor',
 'auto',
 'baby',
 'cool_stuff',
 'garden_tools',
 'telephony',
 'perfumery',
 'toys',
 'office_furniture',
 'stationery',
 'pet_shop',
 'construction_tools_construction',
 'electronics',
 'musical_instruments']

In [52]:
df_2018_20 = []
for rank in df_2018_top20:
    df3 = df_2018[df_2018["product_category_name_english"] == rank]
    df_2018_20.append(df3)

df_2018 = pd.concat(df_2018_20).reset_index(drop=True)
df_2018 = df_2018.groupby(["product_category_name_english", pd.Grouper(key="order_purchase_timestamp", freq="M")])["payment_value"].sum()
df_2018 = pd.DataFrame(df_2018).reset_index()
df_2018 = df_2018.pivot_table("payment_value", "order_purchase_timestamp", "product_category_name_english", aggfunc="sum")
df_2018

product_category_name_english,auto,baby,bed_bath_table,computers_accessories,construction_tools_construction,cool_stuff,electronics,furniture_decor,garden_tools,health_beauty,housewares,musical_instruments,office_furniture,perfumery,pet_shop,sports_leisure,stationery,telephony,toys,watches_gifts
order_purchase_timestamp,,,,,,,,,,,,,,,,,,,,
2018-01-31,41132.68,34134.02,90062.59,92572.25,10936.13,45267.54,21513.92,66234.87,32932.31,81770.02,45087.70,16304.90,24261.25,22893.39,12588.86,100324.81,46208.46,20422.82,23972.44,78914.02
2018-02-28,47381.00,28342.66,71688.31,117051.07,10522.00,34289.79,20048.54,46272.19,32701.26,95624.35,43710.98,10064.38,21625.23,23225.21,11266.80,86467.70,12448.41,18504.32,16272.09,66599.36
2018-03-31,51741.95,46096.28,82227.54,99604.05,10808.65,43049.06,16743.91,64579.07,33478.66,97165.34,44809.89,10155.62,38452.62,23220.03,16107.70,96059.54,15688.86,30233.72,26945.55,104754.31
2018-04-30,55335.10,54043.12,84755.44,67512.35,15161.73,40338.13,14683.74,68533.52,40267.14,103668.28,54543.43,21176.37,26294.45,20291.59,21836.55,76953.56,13820.47,20338.04,27324.06,94469.03
2018-05-31,48150.41,38090.45,84809.00,59003.65,23919.59,34160.94,20469.45,64732.06,42319.55,108976.23,76239.38,20526.78,25366.54,27604.51,14040.08,68105.10,15450.76,23224.85,36398.80,127883.15
2018-06-30,52153.81,36675.02,86774.85,50213.32,25368.25,21825.98,13177.98,53838.53,22170.17,123919.13,69143.67,17970.76,15911.92,30923.92,23592.25,53589.37,16268.72,26288.01,25242.01,93652.51
2018-07-31,50749.02,27109.34,67822.07,49298.47,17498.52,24618.33,11803.19,56852.07,30936.53,119066.39,76801.32,9393.83,16732.73,24170.29,22999.92,63922.63,21259.74,30660.82,19843.22,104600.77
2018-08-31,52376.34,26240.12,74396.68,47823.52,26225.84,18558.20,13030.61,54262.98,20876.70,135260.57,72968.12,11705.45,8386.67,28436.32,28547.70,59640.94,19034.04,41431.39,19777.36,76964.09


## top20 전체비교

In [53]:
layout = {
    'title': 
        {
            'text':'<b>Top20 Category (2017&2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2017-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2017 to Aug. 2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_all.iplot(kind="scatter", layout=layout)

In [54]:
layout = {
    'title': 
        {
            'text':'<b>Top20 Category (2017)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2017-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2017 to Dec. 2017',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2017.iplot(kind="scatter", layout=layout)

In [55]:
layout = {
    'title': 
        {
            'text':'<b>Top20 Category (2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2018-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2018 to Aug. 2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2018.iplot(kind="scatter", layout=layout)

## top5 비교

In [56]:
top5_all = df_all_top20[:5]

layout = {
    'title': 
        {
            'text':'<b>Top5 Category (2017 & 2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2017-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2017 to Aug. 2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_all[top5_all].iplot(kind="scatter", layout=layout )

In [57]:
top5_2017 = df_2017_top20[:5]

layout = {
    'title': 
        {
            'text':'<b>Top5 Category (2017)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2017-01-31",
        'dtick': 'M1',
        'title': {
            'text':'2017',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2017[top5_2017].iplot(kind="scatter", layout=layout )

In [58]:
top5_2018 = df_2018_top20[:5]

layout = {
    'title': 
        {
            'text':'<b>Top5 Category (2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2018-01-31",
        'dtick': 'M1',
        'title': {
            'text':'2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2018[top5_2018].iplot(kind="scatter", layout=layout )

In [59]:
# health_beauty 카테고리를 제외한 나머지 카테고리는 성장성이 둔화하거나 하락하는 추세를 보였음

## new_category 비교

In [60]:
new_category= list(set(df_2018_top20) - set(df_2017_top20))
new_category

['construction_tools_construction', 'musical_instruments', 'electronics']

In [61]:
layout = {
    'title': 
        {
            'text':'<b>New Category (2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2018-01-31",
        'dtick': 'M1',
        'title': {
            'text':'2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2018[new_category].iplot(kind="scatter", theme="white" ,layout=layout)

In [62]:
# 2017 & 2018 전체데이터 new category 추이
df_all_new = df2[df2["year"] != 2016]
df_all_new = df_all_new.groupby(["product_category_name_english", pd.Grouper(key="order_purchase_timestamp", freq="M")])["payment_value"].sum()
df_all_new = pd.DataFrame(df_all_new).reset_index()
df_all_new = df_all_new.pivot_table("payment_value", "order_purchase_timestamp", "product_category_name_english", aggfunc="sum")
df_all_new.head()

product_category_name_english,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,books_imported,...,security_and_services,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,watches_gifts
order_purchase_timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-31,107.76,765.33,NaN,NaN,NaN,5386.38,6941.93,4911.83,271.55,NaN,...,209.06,NaN,1788.25,NaN,10754.92,1754.61,NaN,1145.89,5302.23,8310.83
2017-02-28,331.10,2688.34,NaN,NaN,247.85,14264.10,3544.53,18263.15,950.02,NaN,...,NaN,NaN,8022.25,NaN,18533.57,2646.03,940.52,8369.64,10359.95,11524.20
2017-03-31,110.69,4390.56,255.00,NaN,1426.97,15304.25,4358.31,30112.54,2951.30,NaN,...,NaN,91.23,12033.13,NaN,29284.00,6994.89,587.91,10366.15,14248.29,26092.77
2017-04-30,NaN,2748.32,147.17,NaN,1103.95,17205.68,4690.33,28842.93,1144.97,NaN,...,NaN,183.65,24045.81,NaN,26977.70,5119.99,1312.23,9305.75,15072.16,24958.48
2017-05-31,1763.74,1068.45,7302.75,NaN,2333.19,20227.30,11019.20,38989.73,2199.17,NaN,...,NaN,775.94,5727.35,NaN,39530.31,6664.42,1506.31,12051.75,25370.33,38960.51


In [63]:
layout = {
    'title': 
        {
            'text':'<b>New Category (2017 & 2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2017-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2017 to Aug. 2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_all_new[new_category].iplot(kind="scatter", layout=layout)

In [64]:
# construction_tools_construction 경우 건설업의 성장으로 꾸준한 상승, 나머지 두 카테고리는 2018년 다른 카테고리에 비해 상대적인 소폭하락으로 탑 20에 진입

## - top20 중 상승 카테고리

In [65]:
increased_category = ["health_beauty","housewares","pet_shop","construction_tools_construction","telephony"]

In [66]:
layout = {
    'title': 
        {
            'text':'<b>Increased Category (2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2018-01-31",
        'dtick': 'M1',
        'title': {
            'text':'2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_2018[increased_category].iplot(kind="scatter", theme="white" ,layout=layout)

In [67]:
layout = {
    'title': 
        {
            'text':'<b>Increased Category (2017 & 2018)</b>',
            'font': {
                'size': 20,
                'color': '#37474F'
            },
            'x': 0.5,
            'y': 0.88

        },
    'xaxis': {
        'tick0': "2018-01-31",
        'dtick': 'M1',
        'title': {
            'text':'from Jan. 2017 to Aug. 2018',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
    },
    'yaxis': {
        'title': {
            'text':'sales',
            'font': {
                    'size': 15,
                    'color': '#37474F'
                }
            }
        
    }    
}
df_all_new[increased_category].iplot(kind="scatter" ,layout=layout)


In [68]:
# 정규화

In [69]:
# # 2017년
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaled_values_2017 = scaler.fit_transform(df_2017)
# df_2017.loc[:,:] = scaled_values_2017

In [70]:
# # 2018
# scaler = MinMaxScaler()
# scaled_values_2018 = scaler.fit_transform(df_2018)
# df_2018.loc[:,:] = scaled_values_2017

In [71]:
# # df_all_new
# scaler = MinMaxScaler()
# scaled_values_all_new = scaler.fit_transform(df_all_new)
# df_all_new.loc[:,:] = scaled_values_all_new

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">olist 월별 제품 카테고리 트렌드 분석</font><br>
</div>

In [72]:
# 필요한 컬럼 가져오기
product_order_month_df = df[["product_category_name_english","payment_value","order_purchase_timestamp"]].copy()
product_order_month_df["order_purchase_timestamp"] = product_order_month_df["order_purchase_timestamp"].astype("str")
product_order_month_df["months"] = product_order_month_df["order_purchase_timestamp"].str.split("-",expand=True)[1].astype("int")
product_order_month_df.drop(columns="order_purchase_timestamp", inplace=True)
product_order_month_df.head()

,product_category_name_english,payment_value,months
0,cool_stuff,72.19,9
1,cool_stuff,73.86,6
2,cool_stuff,83.23,5
3,cool_stuff,75.07,8
4,cool_stuff,72.19,8


In [73]:
# 컬럼이름 변경하기
product_order_month_df = product_order_month_df.rename(columns={"product_category_name_english":"category"})
product_order_month_df.head(2)

,category,payment_value,months
0,cool_stuff,72.19,9
1,cool_stuff,73.86,6


In [74]:
# (1) 월별 카테고리 건수 데이터 구하기

In [75]:
product_order_month_count = product_order_month_df.groupby(["months", "category"])["payment_value"].size().reset_index()
product_order_month_count

,months,category,payment_value
0,1,agro_industry_and_commerce,17
1,1,air_conditioning,25
2,1,art,18
3,1,audio,25
4,1,auto,270
...,...,...,...
790,12,stationery,228
791,12,tablets_printing_image,6
792,12,telephony,236
793,12,toys,412


In [76]:
# 월별 max 값 & category 가져오기
product_order_month_max_count = []

for num in range(1,13):
    data = product_order_month_count[product_order_month_count["months"]==num]
    data = data[data["payment_value"]==data["payment_value"].max()]
    product_order_month_max_count.append(data)

product_order_month_max_count = pd.concat(product_order_month_max_count).reset_index(drop=True)
product_order_month_max_count

,months,category,payment_value
0,1,bed_bath_table,725
1,2,computers_accessories,881
2,3,bed_bath_table,884
3,4,bed_bath_table,823
4,5,health_beauty,927
5,6,health_beauty,1016
6,7,bed_bath_table,1000
7,8,health_beauty,1098
8,9,bed_bath_table,450
9,10,bed_bath_table,440


In [77]:
# 컬럼명 바꾸기
product_order_month_max_count.rename(columns={"payment_value":"count"}, inplace=True)
product_order_month_max_count.head(2)

,months,category,count
0,1,bed_bath_table,725
1,2,computers_accessories,881


In [78]:
# month 이름 바꿔주기
def months(x):
    if x == 1:
        x = "Jan"
    elif x == 2:
        x = "Feb"
    elif x == 3:
        x = "Mar"
    elif x == 4:
        x = "Apr"
    elif x == 5:
        x = "May"
    elif x == 6:
        x = "Jun"
    elif x == 7:
        x = "Jul"
    elif x == 8:
        x = "Aug"
    elif x == 9:
        x = "Sep"
    elif x == 10:
        x = "Oct"
    elif x == 11:
        x = "Nov"
    elif x == 12:
        x = "Dec"
    return x

In [79]:
product_order_month_max_count["months"] = product_order_month_max_count["months"].apply(months)
product_order_month_max_count["category_of_month"] = product_order_month_max_count["category"] + "(" + product_order_month_max_count["months"] + ")"
product_order_month_max_count = product_order_month_max_count[["category_of_month","count"]]
product_order_month_max_count

,category_of_month,count
0,bed_bath_table(Jan),725
1,computers_accessories(Feb),881
2,bed_bath_table(Mar),884
3,bed_bath_table(Apr),823
4,health_beauty(May),927
5,health_beauty(Jun),1016
6,bed_bath_table(Jul),1000
7,health_beauty(Aug),1098
8,bed_bath_table(Sep),450
9,bed_bath_table(Oct),440


In [80]:
#(2) 월별 카테고리 금액

In [81]:
product_order_month_price = product_order_month_df.groupby(["months", "category"])["payment_value"].sum().reset_index()
product_order_month_price

,months,category,payment_value
0,1,agro_industry_and_commerce,6404.27
1,1,air_conditioning,5942.85
2,1,art,2698.30
3,1,audio,4233.39
4,1,auto,46519.06
...,...,...,...
790,12,stationery,29413.90
791,12,tablets_printing_image,485.33
792,12,telephony,16388.48
793,12,toys,63352.46


In [82]:
product_order_month_max_price = []

for num in range(1,13):
    data = product_order_month_price[product_order_month_price["months"]==num]
    data = data[data["payment_value"]==data["payment_value"].max()]
    product_order_month_max_price.append(data)

product_order_month_max_price = pd.concat(product_order_month_max_price).reset_index(drop=True)
product_order_month_max_price

,months,category,payment_value
0,1,sports_leisure,111079.73
1,2,computers_accessories,130065.78
2,3,computers_accessories,130890.78
3,4,health_beauty,129167.03
4,5,watches_gifts,166843.66
5,6,health_beauty,160806.31
6,7,health_beauty,158666.05
7,8,health_beauty,190715.03
8,9,bed_bath_table,62014.85
9,10,watches_gifts,72608.08


In [83]:
product_order_month_max_price["months"] = product_order_month_max_price["months"].apply(months)
product_order_month_max_price["category_of_month"] = product_order_month_max_price["category"] + "(" + product_order_month_max_price["months"] + ")"
product_order_month_max_price = product_order_month_max_price[["category_of_month","payment_value"]]
product_order_month_max_price.head(2)

,category_of_month,payment_value
0,sports_leisure(Jan),111079.73
1,computers_accessories(Feb),130065.78


In [84]:
# 시각화

In [85]:
product_order_month_max_count

,category_of_month,count
0,bed_bath_table(Jan),725
1,computers_accessories(Feb),881
2,bed_bath_table(Mar),884
3,bed_bath_table(Apr),823
4,health_beauty(May),927
5,health_beauty(Jun),1016
6,bed_bath_table(Jul),1000
7,health_beauty(Aug),1098
8,bed_bath_table(Sep),450
9,bed_bath_table(Oct),440


In [86]:
colors = ['#03588C',] * len(product_order_month_max_price["category_of_month"])
colors[7] = '#F24472'

In [87]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=product_order_month_max_count["category_of_month"], 
        y=product_order_month_max_count['count'],
        text=product_order_month_max_count['count'], 
        textposition='auto', 
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Category of The Month (count)</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 01 to Dec. 31",
            "showticklabels":True,
            "dtick": 1,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "count"
        },
        "template":'plotly_white'
    }
)
fig.add_annotation(
            x="health_beauty(Aug)",
            y=1120,
            text="<b>Peaked Monthly(Aug)</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77CFD9",
            ax=20,
            ay=-30,
            bordercolor="#77CFD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9
)
fig.show()

In [88]:
# 2) 금액별 그래프

In [89]:
product_order_month_max_price

,category_of_month,payment_value
0,sports_leisure(Jan),111079.73
1,computers_accessories(Feb),130065.78
2,computers_accessories(Mar),130890.78
3,health_beauty(Apr),129167.03
4,watches_gifts(May),166843.66
5,health_beauty(Jun),160806.31
6,health_beauty(Jul),158666.05
7,health_beauty(Aug),190715.03
8,bed_bath_table(Sep),62014.85
9,watches_gifts(Oct),72608.08


In [90]:
colors = ['#03588C',] * len(product_order_month_max_price["category_of_month"])
colors[7] = '#F24472'

In [91]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=product_order_month_max_price["category_of_month"], 
        y=product_order_month_max_price['payment_value'],
        text=product_order_month_max_price['payment_value'], 
        textposition='auto', 
        texttemplate='RS$ %{text:,.0f}',
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Category of The Month (sales)</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 01 to Dec. 31",
            "showticklabels":True,
            "dtick": 1,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "sales"
        },
        "template":'plotly_white'
    }
)
fig.add_annotation(
            x="health_beauty(Aug)",
            y=195715.03,
            text="<b>Peaked Monthly(Aug)</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77CFD9",
            ax=20,
            ay=-30,
            bordercolor="#77CFD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9
)
fig.show()